In [32]:
import keras
import nltk
import pandas as pd
import numpy as np
import re
import codecs
import json

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [33]:
with open('NewYorkState_hotels_reviews.json') as f:
    data = pd.DataFrame([json.loads(line) for line in f])

In [34]:
data.sample(n=5)

,Cleanliness,Comfort,Facilities,FreeWifi,GivenScore,Heading,HotelName,Location,NegativeReview,NumReviewGiven,PositiveReview,SelfTag,Staff,StayDate,UserAgeGroup,UserCountry,UserName,ValueForMoney
197208,[8.2],[8.4],[8.1],[8.3],[\n10\n],[Excellent service!],[Wingate by Wyndham Niagara Falls],[8.8],[I wasn't too impressed by the continental bre...,[\n2 reviews\n],[Very clean rooms!],"[\n, Leisure trip\n, \n, Family with young c...",[8.5],[\nStayed in March 2017\n],[\n],[USA],[Alma],[8]
289596,[8],[7.8],[7.4],NaN,[\n3.8\n],[no room service\r],[The Gregory],[9],[no room service\rtoo expensinve with what you...,[\n3 reviews\n],NaN,"[\n, Leisure trip\n, \n, Couple\n]",[8.1],NaN,[\n],[Lebanon],[Abi],[7.1]
49574,[9.2],[9],[8.6],[9.2],[\n10\n],[Everything!],[The William],[9.3],[No magnifying mirror in the room \rNo microwa...,[\n26 reviews\n],[Everything!],"[\n, Business trip\n, \n, Couple\n]",[8.7],NaN,[\n],[USA],[Susan],[8.2]
199070,[8.6],[8.2],[7.9],[8.1],[\n10\n],[This location is where we spend our 1year Ann...,[Hilton Garden Inn Central Park South],[9.2],NaN,[\n2 reviews\n],[It was so clean and friendly ],"[\n, Couple\n, \n, Queen Room\n, \n, Stayed...",[8.4],[\nStayed in July 2018\n],[\n],[USA],[Natalie],[7.4]
186781,[8.5],[7.9],[7.8],[7],[\n7.9\n],[I loved the staff and their energy during my ...,[The Marcel at Gramercy],[8.7],[the noise of the heat/ac split in the room.],[\n3 reviews\n],[I loved the staff and their energy during my ...,"[\n, Solo traveller\n, \n, Standard Room\n,...",[8.9],[\nStayed in April 2017\n],[\n],[United Arab Emirates],[Ghada],[7.7]


In [35]:
data.shape

(335879, 18)

In [36]:
# data['PositiveReview']

In [37]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()

def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [38]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/argha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

In [40]:
for w in ['dogs', 'ran', 'discouraged']:
    print(w, get_lemma(w), get_lemma2(w))

dogs dog dog
ran run ran
discouraged discourage discouraged


In [41]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/argha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [43]:
all_pos_rev = [line[0] for line in data['PositiveReview'] if str(line)!='nan']
all_pos_rev = [re.sub('/', ' ', sent) for sent in all_pos_rev]
# Remove new line characters
all_pos_rev = [re.sub('\s+', ' ', sent) for sent in all_pos_rev]

# Remove distracting single quotes
all_pos_rev = [re.sub("\'", "", sent) for sent in all_pos_rev]

print(all_pos_rev[:1])

['The hospitality the first day was exceptional. Hotel lost power the next morning, it was very inconvenient in our room trying to get ready to check out. No mention was made at the front desk about the possible customer inconvenience. A little surprised']


In [44]:
len(all_pos_rev)

307265

In [45]:
# def clean_text(text):
#     lower_text = text.apply(lambda x: " ".join(x.lower() for x in x.split()))
#     no_punctuation = lower_text.str.replace('[^\w\s]','')
#     tokenized_text = word_tokenize(text.lower())
#     cleaned_text = [t for t in tokenized_text if t not in STOPWORDS and re.match('[a-zA-Z\-][a-zA-Z\-]{2,}', t)]
#     return cleaned_text

In [46]:
import random
text_data = []
for line in all_pos_rev:
    tokens = prepare_text_for_lda(line)
#     if random.random() > .99:
#         #print(tokens)
    text_data.append(tokens)

In [47]:
len(text_data)

307265

In [48]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [49]:
import gensim
NUM_TOPICS = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')

In [52]:
topics = ldamodel.print_topics(num_words=5)
for topic in topics:
    print(topic)

(18, '0.492*"location" + 0.369*"great" + 0.067*"staff" + 0.027*"facility" + 0.024*"hotel"')
(28, '0.192*"lobby" + 0.065*"morning" + 0.054*"drink" + 0.042*"stuff" + 0.038*"welcome"')
(25, '0.301*"place" + 0.105*"wonderful" + 0.064*"always" + 0.029*"going" + 0.027*"walk"')
(0, '0.058*"every" + 0.058*"hotel" + 0.050*"staff" + 0.042*"visit" + 0.038*"concierge"')
(9, '0.130*"right" + 0.084*"street" + 0.073*"building" + 0.056*"minutes" + 0.049*"state"')
(15, '0.106*"bathroom" + 0.102*"shower" + 0.053*"water" + 0.052*"suite" + 0.051*"kitchen"')
(7, '0.243*"amaze" + 0.213*"love" + 0.051*"enough" + 0.051*"decor" + 0.038*"quality"')
(24, '0.157*"front" + 0.112*"staff" + 0.080*"pleasant" + 0.062*"especially" + 0.055*"expect"')
(17, '0.215*"walking" + 0.182*"distance" + 0.090*"within" + 0.058*"want" + 0.056*"location"')
(19, '0.343*"excellent" + 0.201*"location" + 0.142*"service" + 0.085*"staff" + 0.063*"cleanliness"')
(13, '0.338*"everything" + 0.262*"perfect" + 0.106*"need" + 0.059*"close" + 0.0

In [53]:
new_doc = 'Practical Bayesian Optimization of Machine Learning Algorithms'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(877, 1), (1528, 1), (5104, 1), (9895, 1)]
[(0, 0.011111786), (1, 0.011111786), (2, 0.011111786), (3, 0.011111786), (4, 0.011111786), (5, 0.011111786), (6, 0.011111786), (7, 0.011111786), (8, 0.011111786), (9, 0.011111786), (10, 0.011111786), (11, 0.011111786), (12, 0.3444565), (13, 0.011111786), (14, 0.011111786), (15, 0.011111786), (16, 0.011111786), (17, 0.011111786), (18, 0.011111786), (19, 0.011111786), (20, 0.011111786), (21, 0.011111786), (22, 0.011111786), (23, 0.011111786), (24, 0.011111786), (25, 0.3444135), (26, 0.011111786), (27, 0.011111786), (28, 0.011111786), (29, 0.011111786)]


In [54]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 3, id2word=dictionary, passes=15)
ldamodel.save('model3.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.070*"location" + 0.057*"hotel" + 0.040*"close" + 0.030*"great"')
(1, '0.027*"hotel" + 0.020*"breakfast" + 0.017*"check" + 0.012*"small"')
(2, '0.160*"location" + 0.134*"staff" + 0.088*"clean" + 0.086*"great"')


In [55]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
ldamodel.save('model10.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.111*"value" + 0.082*"like" + 0.066*"money" + 0.052*"cleanliness"')
(1, '0.312*"breakfast" + 0.051*"great" + 0.036*"include" + 0.025*"comfortable"')
(2, '0.044*"check" + 0.040*"bathroom" + 0.038*"clean" + 0.034*"comfortable"')
(3, '0.093*"square" + 0.089*"location" + 0.075*"times" + 0.074*"central"')
(4, '0.097*"everything" + 0.048*"hotel" + 0.040*"staff" + 0.025*"need"')
(5, '0.402*"location" + 0.190*"great" + 0.079*"excellent" + 0.073*"staff"')
(6, '0.125*"hotel" + 0.045*"would" + 0.043*"great" + 0.041*"place"')
(7, '0.205*"staff" + 0.144*"clean" + 0.139*"friendly" + 0.102*"helpful"')
(8, '0.087*"location" + 0.069*"close" + 0.050*"restaurant" + 0.039*"convenient"')
(9, '0.052*"coffee" + 0.042*"lobby" + 0.039*"nothing" + 0.028*"hotel"')


## pyLDAvis

In [56]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

In [1]:
import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

NameError: name 'lda' is not defined

In [58]:
lda3 = gensim.models.ldamodel.LdaModel.load('model3.gensim')
lda_display3 = pyLDAvis.gensim.prepare(lda3, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display3)

/home/argha/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [59]:
lda10 = gensim.models.ldamodel.LdaModel.load('model10.gensim')
lda_display10 = pyLDAvis.gensim.prepare(lda10, corpus, dictionary, sort_topics=False)
pyLDAvis.display(lda_display10)

/home/argha/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))
